## Consumer Complaints Dataset
### 1. Preprocessing
The categories ('Product' field) of the complaints are often subsets of one another, for example data under the product 'Credit card' could also be classified under 'Credit Card, Prepaid Card'. We merge data of this type together to create fewer Products but more Sub-products. We would like to predict sub-products once the initial pass is done.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import re
import numpy as np
import pandas as pd

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
def gen_model_data(df, subset=None, features=None):
    """
    Drop rows with NaNs in 'Product', 'Issue', 'Date' 
    and 'Consumer complaint narrative' columns.
    """
    df = df.dropna(subset=subset)
    if features:
        df = df[features]
    return df

In [6]:
complaints_file = '../data/Consumer_Complaints.csv'

In [7]:
df = pd.read_csv(complaints_file, low_memory=False)

In [8]:
subset = ['Product', 'Issue', 'Consumer complaint narrative']
df = gen_model_data(df, subset=subset)

### Merge some of the fields together

In [9]:
df.to_csv('../data/with_narrative.csv', index=False)

In [10]:
df.Product.value_counts()

Credit reporting, credit repair services, or other personal consumer reports    116993
Debt collection                                                                  95954
Mortgage                                                                         57037
Credit reporting                                                                 31588
Credit card or prepaid card                                                      26383
Student loan                                                                     23414
Credit card                                                                      18838
Checking or savings account                                                      15851
Bank account or service                                                          14885
Consumer Loan                                                                     9473
Vehicle loan or lease                                                             6837
Money transfer, virtual currency, or money 

In [11]:
def merge_products(df):
    # Merge Payday loan with other loans
    main_type = 'Payday loan, title loan, or personal loan'
    subset_type = 'Payday loan'
    subset = df[df.Product == subset_type]
    subset['Sub-product'] = subset_type
    subset['Product'] = main_type
    main_set = df[df.Product == main_type]
    loans = pd.concat([subset, main_set])
    
    # Credit card
    main_type = 'Credit card or prepaid card'
    subset_type = 'Credit card'
    subset = df[df.Product == subset_type]
    subset['Sub-product'] = subset_type
    subset['Product'] = main_type
    main_set = df[df.Product == main_type]
    credit = pd.concat([subset, main_set])

    subset = df[df.Product == 'Prepaid card']
    credit = pd.concat([subset, credit])
    credit['Product'] = main_type
    
    # Merge vehicle loan with consumer loans
    main_type = 'Consumer Loan'
    subset_type = 'Vehicle loan or lease'
    vehicles = df[df.Product == subset_type]

    loan_types = []
    for old, new in [('Loan', 'Vehicle loan'), ('Lease', 'Vehicle lease'), ('Title loan', 'Title loan')]:
        idx = vehicles[(vehicles['Sub-product'] == old)]
        idx['Sub-product'] = new
        loan_types.append(idx)
    vehicles = pd.concat(loan_types)

    consumer_loans = df[df.Product == main_type]
    consumer_loans = pd.concat([vehicles, consumer_loans])
    consumer_loans['Product'] = main_type
    
    
    # Merge virtual currency and Money transfer
    main_type = 'Money transfer, virtual currency, or money service'
    subset_type1 = 'Virtual currency'
    subset_type2 = 'Money transfers'
    transfer = df[df.Product == main_type]
    subset1 = df[df.Product == subset_type1]
    subset2 = df[df.Product == subset_type2]

    transfer = pd.concat([subset1, subset2, transfer])
    transfer['Product'] = main_type
    
    # Reporting
    main_type = 'Credit reporting, credit repair services, or other personal consumer reports'
    subset_type = 'Credit reporting'
    subset = df[df.Product == subset_type]
    subset['Sub-product'] = subset_type
    subset['Product'] = main_type
    main_set = df[df.Product == main_type]
    reporting = pd.concat([subset, main_set])
    
    # Banking service
    main_type = 'Bank account or service'
    subset_type = 'Checking or savings account'
    main_set = df[df.Product == main_type]
    subset = df[df.Product == subset_type]

    banking = pd.concat([main_set, subset])
    banking['Product'] = main_type
    
    not_modified = df[df.Product.isin(['Debt collection', 'Mortgage', 'Student loan','Other financial service'])]
    return pd.concat([not_modified, loans, credit, consumer_loans, transfer, reporting, banking])

In [13]:
merged = merge_products(df)

In [14]:
merged['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    148581
Debt collection                                                                  95954
Mortgage                                                                         57037
Credit card or prepaid card                                                      46671
Bank account or service                                                          30736
Student loan                                                                     23414
Consumer Loan                                                                    16310
Money transfer, virtual currency, or money service                                8014
Payday loan, title loan, or personal loan                                         7028
Other financial service                                                            292
Name: Product, dtype: int64

In [15]:
abbrev_map = {'Credit reporting, credit repair services, or other personal consumer reports': 'CR',
                'Debt collection': 'DC', 
                'Mortgage': 'MO', 
                'Credit card or prepaid card': 'CC',
                'Bank account or service': 'BS', 
                'Student loan': 'SL', 
                'Consumer Loan': 'CL', 
                'Money transfer, virtual currency, or money service' :'MT', 
                'Payday loan, title loan, or personal loan': 'PL', 
                'Other financial service': 'OT'}

In [16]:
def map_abbrev(val):
    return abbrev_map[val]

In [17]:
merged['Abbrev'] = merged['Product'].apply(map_abbrev)

In [18]:
merged['Product'].value_counts()

Credit reporting, credit repair services, or other personal consumer reports    148581
Debt collection                                                                  95954
Mortgage                                                                         57037
Credit card or prepaid card                                                      46671
Bank account or service                                                          30736
Student loan                                                                     23414
Consumer Loan                                                                    16310
Money transfer, virtual currency, or money service                                8014
Payday loan, title loan, or personal loan                                         7028
Other financial service                                                            292
Name: Product, dtype: int64

In [19]:
merged.to_csv('../data/product_merged.csv', index=False)